<a href="https://colab.research.google.com/github/arutraj/.githubcl/blob/main/03_transformation_and_regularization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

rs = np.random.RandomState(seed=666)

# Let's create some fake data, just for practice
n =2170
p = 3

X = rs.normal(loc=1, size=(n, p))

y = np.exp(1 + X.sum(axis=1) + rs.normal(size=n))

df = pd.DataFrame({f"X{i+1}": X[:, i] for i in range(p)}).assign(y=y)
df

,X1,X2,X3,y
0,1.824188,1.479966,2.173468,277.380263
1,1.909048,0.428279,0.890503,70.364019
2,1.019028,0.056239,1.640573,43.615190
3,0.213557,1.608870,0.068988,24.228528
4,1.978222,0.263082,0.701267,63.488862
...,...,...,...,...
2165,-0.200493,1.099089,-0.044460,28.673984
2166,0.226775,-0.635790,1.265349,3.924129
2167,1.299628,0.898471,0.853888,208.008840
2168,2.139976,0.741709,1.410680,333.372556


In [2]:
import statsmodels.formula.api as smf

# Vanilla model without any transformations
model = smf.ols(formula="y ~ X1 + X2 + X3", data=df)
fitted_model = model.fit()
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.167
Model:                            OLS   Adj. R-squared:                  0.166
Method:                 Least Squares   F-statistic:                     144.8
Date:                Sun, 10 Nov 2024   Prob (F-statistic):           1.63e-85
Time:                        19:11:22   Log-Likelihood:                -18551.
No. Observations:                2170   AIC:                         3.711e+04
Df Residuals:                    2166   BIC:                         3.713e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -605.3992     52.965    -11.430      0.000    -709.267    -501.532
X1           320.4834     26.305     12.184      0.000     268.899     372.068
X2           326.3211     27.078     12.051      0.000     273.220     379.422
X3           326.4947     26.899     12.138      0.000     273.744     379.245
==============================================================================
Omnibus:                     3630.605   Durbin-Watson:                   2.038
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          2574062.511
Skew:                          11.100   Prob(JB):                         0.00
Kurtosis:                     170.261   Cond. No.                         4.67
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Transformations

## `log` transformations

In [4]:
# Note that y is strictly positive
df.y.min()

0.06511476287477425

In [5]:
# But the model can still predict negative values
new_df = pd.DataFrame({
    "X1": [-1],
    "X2": [-1],
    "X3": [-1],
})
fitted_model.predict(new_df)

,0
0,-1578.698322


In [6]:
# With statsmodels, we can specify the transformation inside the formula

transformed_df = df.assign(log_y = np.log(df.y))

model = smf.ols(formula="log_y ~ X1 + X2 + X3", data=transformed_df)
fitted_model = model.fit()
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  log_y   R-squared:                       0.757
Model:                            OLS   Adj. R-squared:                  0.756
Method:                 Least Squares   F-statistic:                     2245.
Date:                Sun, 10 Nov 2024   Prob (F-statistic):               0.00
Time:                        19:12:18   Log-Likelihood:                -3036.0
No. Observations:                2170   AIC:                             6080.
Df Residuals:                    2166   BIC:                             6103.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.0121      0.042     24.340      0.000       0.931       1.094
X1             1.0309      0.021     49.919      0.000       0.990       1.071
X2             1.0062      0.021     47.329      0.000       0.964       1.048
X3             0.9702      0.021     45.941      0.000       0.929       1.012
==============================================================================
Omnibus:                        0.759   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.684   Jarque-Bera (JB):                0.822
Skew:                          -0.027   Prob(JB):                        0.663
Kurtosis:                       2.922   Cond. No.                         4.67
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
# Now the prediction is log(y), so we need to transform to original scale
np.exp(fitted_model.predict(new_df))

,0
0,0.135987


## Centering/standardization

Similarly, we can create new features of `X` by creating columns with the required transformations.

In [8]:
transformed_df = transformed_df.assign(
    centered_x1 = lambda d: (d.X1 - d.X1.mean())/d.X1.std(ddof=0)
)

model = smf.ols(formula="log_y ~ centered_x1 + X2 + X3", data=transformed_df)
fitted_model = model.fit()
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  log_y   R-squared:                       0.757
Model:                            OLS   Adj. R-squared:                  0.756
Method:                 Least Squares   F-statistic:                     2245.
Date:                Sun, 10 Nov 2024   Prob (F-statistic):               0.00
Time:                        19:13:20   Log-Likelihood:                -3036.0
No. Observations:                2170   AIC:                             6080.
Df Residuals:                    2166   BIC:                             6103.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       1.9879      0.036     54.495      0.000       1.916       2.059
centered_x1     1.0530      0.021     49.919      0.000       1.012       1.094
X2              1.0062      0.021     47.329      0.000       0.964       1.048
X3              0.9702      0.021     45.941      0.000       0.929       1.012
==============================================================================
Omnibus:                        0.759   Durbin-Watson:                   1.991
Prob(Omnibus):                  0.684   Jarque-Bera (JB):                0.822
Skew:                          -0.027   Prob(JB):                        0.663
Kurtosis:                       2.922   Cond. No.                         3.71
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Side-note:

Note the value of the intercept when we fit a model with both the outcome (`y`) and features (`X`) centered.

In [9]:
transformed_df = transformed_df.assign(
    centered_y = lambda d: (d.y - d.y.mean())/d.y.std(ddof=0)
)

model = smf.ols(formula="centered_y ~ centered_x1", data=transformed_df)
fitted_model = model.fit()

fitted_model.params.Intercept

-3.06287113727155e-17

# Regularized model

`statsmodels` provides pre-packaged solutions for solving "regularized" versions of OLS. (so does `sklearn`, but we'll get into that later)

For `statemodels`, you can use the `fit_regularized` instead of `fit`, where the `L1_wt` argument determines the _weight_ between L1 (Lasso) and L2 (Ridge) penalties.

In [12]:
model.fit_regularized

<bound method OLS.fit_regularized of <statsmodels.regression.linear_model.OLS object at 0x7d7ab38e5660>>

From the documentation, note, `fit_regularized` changes the objective of the regression (roughly) to:

$$\mathtt{SSE} + \mathtt{alpha}*((1-\mathtt{L1\_wt})*0.5*\sum\vert\beta\vert^2 + \mathtt{L1\_wt}*\sum\vert params\vert)$$

To be more precise, the first term is some `0.5*SSE/n`, where the `0.5` is a common "trick" used in optimization to cancel out the "squared" value when taking a derivative (e.g., $\frac{d}{dx}x^2=2x$ so $\frac{d}{dx}0.5x^2=x$.)

Then, the $\mathtt{alpha}$ is the weight of the overall penalty (what we've denoted as $\lambda$ in the slides), and the $\mathtt{L1\_wt}$ parameter is a value between 0 and 1 where
if `L1_wt=1` the penalty simplifies to the Lasso penalty,
and if `L1_wt=0` it's the L2 Ridge penalty.

In [13]:
# L1_wt = 1 is the Lasso
fitted_lasso = model.fit_regularized(alpha=0.5, L1_wt=1)

In [14]:
# L1_wt = 0 is Ridge
fitted_ridge = model.fit_regularized(alpha=0.5, L1_wt=0)

We'll get into more detail on how we might find the "best" values of alpha and/or `L1_wt` in the next "Prediction" sections.

In [15]:
fitted_lasso

In [16]:
fitted_ridge